In [1]:
import shapefile
import re
import pandas as pd
import geopy.distance
from tqdm import tqdm_notebook
from shapely.geometry import Polygon

def getLLDist(row, coord1):
    coord2 = (row["Lat"], row["Lon"])
    return (geopy.distance.vincenty(coord1, coord2).km, row["City"])

def getClosestCity(row):
    coord1 = (row["Lat"], row["Lon"])
    return min(cityDf.apply(getLLDist, coord1=coord1, axis=1))

blockDf = pd.read_csv("block-city-mapping.csv")
cityDf = pd.read_csv("city-basics_iowa.csv")

blockDf["Census Block ID"] = blockDf["Census Block ID"].apply(lambda x: str(x))

censusDict = blockDf[["Census Block ID", "City"]].set_index("Census Block ID").to_dict()["City"]
latDict = blockDf[["City", "Lat"]].set_index("City").to_dict()["Lat"]
lonDict = blockDf[["City", "Lon"]].set_index("City").to_dict()["Lon"]

shape = shapefile.Reader("gz_2010_19_150_00_500k.shp")

columns = ["Census Block ID", "Lat", "Lon"]

df = pd.DataFrame(columns=columns)

for feature in shape.shapeRecords():
    blockId = re.sub("[\d]+US", "", feature.record[0])
    if (feature.shape.__geo_interface__["type"] is "MultiPolygon"):
        centerX = sum([p[0] for p in feature.shape.__geo_interface__["coordinates"][0][0]])/len(feature.shape.__geo_interface__["coordinates"][0][0])
        centerY = sum([p[1] for p in feature.shape.__geo_interface__["coordinates"][0][0]])/len(feature.shape.__geo_interface__["coordinates"][0][0])
        centroid = (centerX, centerY)
    else:
        centerX = sum([p[0] for p in feature.shape.__geo_interface__["coordinates"][0]])/len(feature.shape.__geo_interface__["coordinates"][0])
        centerY = sum([p[1] for p in feature.shape.__geo_interface__["coordinates"][0]])/len(feature.shape.__geo_interface__["coordinates"][0])
        centroid = (centerX, centerY)

    df.loc[len(df)] = [blockId, centroid[1], centroid[0]]

df["Closest City"] = df.apply(getClosestCity, axis=1)

In [2]:
compCity = "Correctionville"

def getDistanceToCity(row):
    city = compCity
    cityLat = float(cityDf[cityDf["City"].str.contains(city)]["Lat"])
    cityLon = float(cityDf[cityDf["City"].str.contains(city)]["Lon"])
    coord1 = (cityLat, cityLon)
    coord2 = (row["Lat"], row["Lon"])
    return geopy.distance.vincenty(coord1, coord2).km
    
    
df["Dist to Correctionville"] = df.apply(getDistanceToCity, axis=1)
df.sort_values(by=["Dist to Correctionville"])

,Census Block ID,Lat,Lon,Closest City,Dist to Correctionville
1580,191930031002,42.500867,-95.808594,"(3.32824558691, Correctionville, IA)",3.328246
1582,191930031004,42.425487,-95.750214,"(6.35475065913, Correctionville, IA)",6.354751
1581,191930031003,42.415241,-95.846604,"(3.45807754205, Anthon, IA)",8.513661
1579,191930031001,42.510450,-95.907906,"(4.99159115394, Pierson, IA)",10.799689
1647,191930031005,42.317275,-95.819747,"(7.18652607149, Oto, IA)",17.939025
1596,190930901002,42.459707,-95.566167,"(3.57188514956, Holstein, IA)",18.072409
1595,190930901001,42.517657,-95.555292,"(3.59911843333, Holstein, IA)",19.401343
287,191499705001,42.590277,-95.974359,"(0.50988667931, Kingsley, IA)",20.049723
974,190350803002,42.655370,-95.728600,"(7.59319956886, Quimby, IA)",20.378646
1032,190350804001,42.621846,-95.616134,"(2.35438480192, Quimby, IA)",21.255658
